## Download the data

In [1]:
import numpy as np
import os
import tensorflow as tf
import pandas as pd

###### Do not modify here ###### 

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

# training on MNIST but only on digits 0 to 4
X_train1 = mnist.train.images[mnist.train.labels < 5]
y_train1 = mnist.train.labels[mnist.train.labels < 5]
X_valid1 = mnist.validation.images[mnist.validation.labels < 5]
y_valid1 = mnist.validation.labels[mnist.validation.labels < 5]
X_test1 = mnist.test.images[mnist.test.labels < 5]
y_test1 = mnist.test.labels[mnist.test.labels < 5]
###### Do not modify here ###### 


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


## Get the data shape

In [2]:
print('Train', mnist.train.num_examples, 
      'Validation', mnist.validation.num_examples,
      'Test', mnist.test.num_examples)

print('Train images :', X_train1.shape,
      'Labels :', y_train1.shape)

print('Validation images :', X_valid1.shape,
      'Labels :', y_valid1.shape)

print('Test images :', X_test1.shape,
      'Labels :', y_test1.shape)


Train 55000 Validation 5000 Test 10000
Train images : (28038, 784) Labels : (28038,)
Validation images : (2558, 784) Labels : (2558,)
Test images : (5139, 784) Labels : (5139,)


## One-hot encoded y values

In [3]:
y_train_one_hot = np.zeros((len(y_train1), 5))
y_train_one_hot[np.arange(len(y_train1)), y_train1] = 1

y_valid_one_hot = np.zeros((len(y_valid1), 5))
y_valid_one_hot[np.arange(len(y_valid1)), y_valid1] = 1

y_test_one_hot = np.zeros((len(y_test1), 5))
y_test_one_hot[np.arange(len(y_test1)), y_test1] = 1

print('Check some examples of one-hot encoding:')
for i in range(10):
    print(y_train1[i], y_train_one_hot[i], '\n')

print ("Train Label: ", y_train_one_hot.shape)
print ("Validation Label: ", y_valid_one_hot.shape)
print ("Test Label: ", y_test_one_hot.shape)


Check some examples of one-hot encoding:
3 [ 0.  0.  0.  1.  0.] 

4 [ 0.  0.  0.  0.  1.] 

1 [ 0.  1.  0.  0.  0.] 

1 [ 0.  1.  0.  0.  0.] 

0 [ 1.  0.  0.  0.  0.] 

0 [ 1.  0.  0.  0.  0.] 

3 [ 0.  0.  0.  1.  0.] 

1 [ 0.  1.  0.  0.  0.] 

2 [ 0.  0.  1.  0.  0.] 

0 [ 1.  0.  0.  0.  0.] 

Train Label:  (28038, 5)
Validation Label:  (2558, 5)
Test Label:  (5139, 5)


### Construct the layer function

In [4]:
def layer(output_dim, input_dim, inputs, layer, dropout_rate, activation=None):

    W = tf.get_variable("W" + layer, shape=[input_dim, output_dim], initializer=tf.contrib.layers.xavier_initializer())
    b = tf.get_variable("b" + layer, shape=[1, output_dim], initializer=tf.contrib.layers.xavier_initializer())

    XWb = tf.matmul(inputs, W) + b
    XWb = tf.nn.dropout(XWb, (1-dropout_rate))
    if activation is None:
        outputs = XWb
    else:
        outputs = activation(XWb)
    return outputs


## contruct the input layer

In [5]:
x = tf.placeholder("float", [None, 784])


## contruct the hidden layers

In [6]:
h1 = layer(output_dim=128, input_dim=784, inputs=x, layer="h1", dropout_rate=0, activation=tf.nn.elu)
h2 = layer(output_dim=128, input_dim=128, inputs=h1, layer="h2", dropout_rate=0, activation=tf.nn.elu)
h3 = layer(output_dim=128, input_dim=128, inputs=h2, layer="h3", dropout_rate=0, activation=tf.nn.elu)
h4 = layer(output_dim=128, input_dim=128, inputs=h3, layer="h4", dropout_rate=0, activation=tf.nn.elu)
h5 = layer(output_dim=128, input_dim=128, inputs=h4, layer="h5", dropout_rate=0, activation=tf.nn.elu)


## contruct the output layer

In [7]:
y_predict = layer(output_dim=5, input_dim=128, inputs=h5, layer="output", dropout_rate=0, activation=None)
y_label = tf.placeholder("float", [None, 5])


## start training

In [8]:
# import sklearn Kfold to implement cross-validation
from sklearn.model_selection import KFold

# hyper parameters
lr = 0.005
batch_size = 256
epochs = 350
saturate_limit = 20  # for applying early stopping
fold_num = 10

# parameters
saturate_count = 0
best_acc = 0.
best_epoch = -1


# set the loss function; tf.nn.sparse_softmax_cross_entropy_with_logits required by the homework spec
loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_predict, labels=y_label))

# set the optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=lr) \
                    .minimize(loss_function)

# compare prediction and label
correct_prediction = tf.equal(tf.argmax(y_label, 1), tf.argmax(y_predict, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

# Merge the train and validation list
X_data = np.concatenate((X_train1, X_valid1), axis=0)
y_data = np.concatenate((y_train_one_hot, y_valid_one_hot), axis=0)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # 10-folder
    kf = KFold(n_splits=fold_num)
    fold_flag = 1
    for train_index, test_index in kf.split(X_data):
        X_train_tmp, X_valid_tmp = X_data[train_index], X_data[test_index]
        y_train_tmp, y_valid_tmp = y_data[train_index], y_data[test_index]
        
        for e in range(epochs):
            for i in range(int(X_train_tmp.shape[0] / batch_size)):
                if(i==0):
                    batch_x = X_train_tmp[:batch_size]
                    batch_y = y_train_tmp[:batch_size]
                else:
                    batch_x = X_train_tmp[i * batch_size : (i + 1) * batch_size]
                    batch_y = y_train_tmp[i * batch_size : (i + 1) * batch_size]
                    
                sess.run(optimizer, feed_dict={x: batch_x, y_label: batch_y})
        
            # validate
            loss, acc = sess.run([loss_function, accuracy], feed_dict={x: X_valid_tmp, y_label: y_valid_tmp})
        
            if best_acc < acc:
                best_acc = acc
                best_epoch = e
                saturate_count = 0
            else:
                saturate_count += 1
                if saturate_count >= saturate_limit:  # stop if saturate
                    break
    
        
        # print the best epoch results   
        print('*' * 60)
        print('fold: ', fold_flag)
        print('Best epoch:', best_epoch)
        print('Best accurancy:', best_acc)
        print('*' * 60, '\n')
        fold_flag = fold_flag + 1
    
    # print the test data accurancy
    print('=' * 60)
    print("Test data accurancy", sess.run(accuracy,
                               feed_dict={x: X_test1,
                                          y_label: y_test_one_hot}))
    print('=' * 60, '\n')
    
    # get prediction results
    predict_result = sess.run(tf.argmax(y_predict, 1), 
                                 feed_dict={x: X_test1})
    
    # count the true or false num of label
    recall_matrix = np.zeros((5, 2))
    precision_matrix = np.zeros((5, 2))
    compare_result = sess.run(tf.equal(predict_result, y_test1))
   
    # construct the confusion matrix
    for i in range(len(predict_result)):
        if(compare_result[i]==True):
            recall_matrix[y_test1[i]][0] += 1
            precision_matrix[predict_result[i]][0] += 1
        else:
            recall_matrix[y_test1[i]][1] += 1
            precision_matrix[predict_result[i]][1] += 1
            
    # count the recall and precision value for every label
    label_dict = []
    for i in range(5):
        tmp_obj = {}
        tmp_obj['label'] = i
        tmp_obj['Precision'] = i
        pre =  precision_matrix[i][0]/(precision_matrix[i][0] + precision_matrix[i][1])
        rec = recall_matrix[i][0]/(recall_matrix[i][0] + recall_matrix[i][1])
        tmp_obj['Precision'] = pre
        tmp_obj['Recall'] = rec
        label_dict.append(tmp_obj)
    label_df = pd.DataFrame(label_dict)
        
    

************************************************************
fold:  1
Best epoch: 11
Best accurancy: 0.989542
************************************************************ 

************************************************************
fold:  2
Best epoch: 0
Best accurancy: 0.99902
************************************************************ 

************************************************************
fold:  3
Best epoch: 0
Best accurancy: 0.99902
************************************************************ 

************************************************************
fold:  4
Best epoch: 0
Best accurancy: 0.99902
************************************************************ 

************************************************************
fold:  5
Best epoch: 0
Best accurancy: 0.99902
************************************************************ 

************************************************************
fold:  6
Best epoch: 0
Best accurancy: 1.0
***************************************

### Show the Precision and recall by pandas

In [9]:
label_df[['label', 'Precision', 'Recall']]

label  Precision    Recall
0      0   0.992872  0.994898
1      1   0.999112  0.991189
2      2   0.992149  0.979651
3      3   0.991115  0.994059
4      4   0.978979  0.995927

In [10]:
# First, We turn the y to one_hot 
# We construct five hidden layer and the activation function use the "tf.nn.elu"
# Our epochs value = 350, but we have the early stop mechanism
# And our batch size = 256(X_train1), we make every bath to train
# We give the learning rate = 0.005, and we want to find the minimize loss value (AdamOptimizer)
# And then, we use the validation date to adjust our model.
# Our best accurancy could achieve "0.995"
# We use the X_test1 data to give our model, and get the value is close to epoch accurancy value. 
# "So our model is not overfitting."
# We use the X_test1 data to get the prediction results, and count the precission and recall value.

# Bonus part 
# We implement the cross-validation and dropout
# We use the scikit learn's KFold to implement
# The cross-validation method make the accurancy higher 
# But the value is near 1, we guess that the class of label is 0~4
# This range is not very big, so it could get the correct value easily
# But the dropout method couldn't make result better
# So we use another cell to present
# Please follow below


### Implement Dropout

In [11]:
# add the dropout function at the hidden layer
h1_drop = layer(output_dim=128, input_dim=784, inputs=x, layer="h1_drop", dropout_rate=0.1, activation=tf.nn.elu)
h2_drop = layer(output_dim=128, input_dim=128, inputs=h1_drop, layer="h2_drop", dropout_rate=0.1, activation=tf.nn.elu)
h3_drop = layer(output_dim=128, input_dim=128, inputs=h2_drop, layer="h3_drop", dropout_rate=0.1, activation=tf.nn.elu)
h4_drop = layer(output_dim=128, input_dim=128, inputs=h3_drop, layer="h4_drop", dropout_rate=0.1, activation=tf.nn.elu)
h5_drop = layer(output_dim=128, input_dim=128, inputs=h4_drop, layer="h5_drop", dropout_rate=0.1, activation=tf.nn.elu)
y_predict_drop = layer(output_dim=5, input_dim=128, inputs=h5_drop, layer="output_drop", dropout_rate=0.1, activation=None)

# hyper parameters
lr = 0.005
batch_size = 256
epochs = 350
saturate_limit = 20  # for applying early stopping

# parameters
iterations = int(X_train1.shape[0] / batch_size)
saturate_count = 0
best_acc = 0.
best_epoch = -1

# set the loss function; tf.nn.sparse_softmax_cross_entropy_with_logits required by the homework spec
loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_predict_drop, labels=y_label))

# set the optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=lr) \
                    .minimize(loss_function)

correct_prediction = tf.equal(tf.argmax(y_label, 1), tf.argmax(y_predict_drop, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for e in range(epochs):
        for i in range(iterations):
            if(i==0):
                batch_x = X_train1[:batch_size]
                batch_y = y_train_one_hot[:batch_size]
            else:
                batch_x = X_train1[i * batch_size : (i + 1) * batch_size]
                batch_y = y_train_one_hot[i * batch_size : (i + 1) * batch_size]

            sess.run(optimizer, feed_dict={x: batch_x, y_label: batch_y})
        
        # validate
        loss, acc = sess.run([loss_function, accuracy], feed_dict={x: X_valid1, y_label: y_valid_one_hot})

        
        if best_acc < acc:
            best_acc = acc
            best_epoch = e
            saturate_count = 0
        else:
            saturate_count += 1
            if saturate_count >= saturate_limit:  # stop if saturate
                break
                    
    print('*' * 60)
    print('Best epoch:', best_epoch)
    print('Best accurancy:', best_acc)
    print('*' * 60, '\n')
    
    print('=' * 60)
    print("Accurancy", sess.run(accuracy,
                               feed_dict={x: X_test1,
                                          y_label: y_test_one_hot}))
    print('=' * 60, '\n')
    
    #get prediction results
    predict_result = sess.run(tf.argmax(y_predict_drop, 1), 
                                 feed_dict={x: X_test1})
    
    # count the true or false num of label
    recall_matrix = np.zeros((5, 2))
    precision_matrix = np.zeros((5, 2))
    compare_result = sess.run(tf.equal(predict_result, y_test1))
   
    for i in range(len(predict_result)):
        if(compare_result[i]==True):
            recall_matrix[y_test1[i]][0] += 1
            precision_matrix[predict_result[i]][0] += 1
        else:
            recall_matrix[y_test1[i]][1] += 1
            precision_matrix[predict_result[i]][1] += 1
    
    label_dict = []
    for i in range(5):
        tmp_obj = {}
        tmp_obj['label'] = i
        tmp_obj['Precision'] = i
        pre =  precision_matrix[i][0]/(precision_matrix[i][0] + precision_matrix[i][1])
        rec = recall_matrix[i][0]/(recall_matrix[i][0] + recall_matrix[i][1])
        tmp_obj['Precision'] = pre
        tmp_obj['Recall'] = rec
        label_dict.append(tmp_obj)
    label_df = pd.DataFrame(label_dict)
        
    


************************************************************
Best epoch: 18
Best accurancy: 0.973417
************************************************************ 

Accurancy 0.969839



### dropout results

In [12]:
label_df[['label', 'Precision', 'Recall']]

label  Precision    Recall
0      0   0.954902  0.993878
1      1   0.979878  0.986784
2      2   0.958654  0.966085
3      3   0.986775  0.960396
4      4   0.990556  0.961303

In [13]:
# We give the dropout_rate=0.1, 
# But we can't get the better accurancy (Normal=0.99, dropout=0.97)
# Maybe the label class is 0~4, it not big.
# So We guess it could influence our results' value
